# Detectar contenido explícito, sugestivo y violento usando Amazon Rekognition

Este notebook ofrece un recorrido por las [API de moderación de contenido](https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html) en Amazon Rekognition. Podemos identificar contenido inapropiado en nuestras bibliotecas de vídeo e imágenes.

## Referencias

- https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectModerationLabels.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetContentModeration.html

## Check setup

In [ ]:
%store -r setup_dependencies_passed

In [ ]:
%store -r setup_dependencies_passed

try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK] Dependencies correctly set up")

## Setup notebook

In [ ]:
import boto3
import sagemaker
from IPython.display import Image as IImage, display
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name
rekognition = boto3.client("rekognition")
s3 = boto3.client("s3")

## Obtener la imagen y subirla a S3

In [ ]:
!mkdir -p ./tmp
!curl -s -o ./tmp/weapon0.jpg "https://collection.nam.ac.uk/images/960/102000-102999/102958.jpg"
!curl -s -o ./tmp/weapon1.jpg "https://observer.com/wp-content/uploads/sites/2/2020/02/56161324_434979100380682_5588907875608035328_n-e1580760304964.png"

In [ ]:
temp_folder = "tmp/"
image_names = [
    "weapon0.jpg",
    "weapon1.jpg"
]

for image in image_names:
    s3.upload_file(
        f"{temp_folder}{image}",
        bucket,
        image
    )

display(IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": image_names[0]})))
display(IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": image_names[1]})))

# Call Rekognition to Detect Unsafe Objects in the Image
Call Amazon Rekognition to detect unsafe content in the image: https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html


In [ ]:
detectModerationLabelsResponse_1 = rekognition.detect_moderation_labels(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name": image_names[1],
        }
    }
)

# Review JSON Response from Rekognition Moderation API

In the JSON response below, you will see Moderation Labels, confidence score and additional information.


In [ ]:
display(detectModerationLabelsResponse_1)

# Display list of detected moderation labels

In [ ]:
for label in detectModerationLabelsResponse_1["ModerationLabels"]:
    print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
    print("  - Parent: {}".format(label["ParentName"]))

# Release Resources

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>